In [1]:
import numpy as np
import pandas as pd
from sklearn import tree
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.preprocessing import OneHotEncoder

In [2]:
df_weather = pd.read_csv('weatherAUS.csv')

In [3]:
for i in range (0,df_weather.shape[1]):
    if type(df_weather[(df_weather.columns)[i]][0]) == str:
        df_weather[(df_weather.columns)[i]].fillna((df_weather[(df_weather.columns)[i]].mode())[0], inplace = True)
    else:
        df_weather[(df_weather.columns)[i]].fillna(df_weather[(df_weather.columns)[i]].mean(), inplace = True)


In [4]:
features = df_weather.columns
attributes = features[:-1]
target = features[-1]

In [5]:
X = df_weather[attributes]
X

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday
0,2008-12-01,Albury,13.4,22.900000,0.6,5.468232,7.611178,W,44.00000,W,...,24.0,71.0,22.0,1007.7,1007.1,8.000000,4.50993,16.9,21.8,No
1,2008-12-02,Albury,7.4,25.100000,0.0,5.468232,7.611178,WNW,44.00000,NNW,...,22.0,44.0,25.0,1010.6,1007.8,4.447461,4.50993,17.2,24.3,No
2,2008-12-03,Albury,12.9,25.700000,0.0,5.468232,7.611178,WSW,46.00000,W,...,26.0,38.0,30.0,1007.6,1008.7,4.447461,2.00000,21.0,23.2,No
3,2008-12-04,Albury,9.2,28.000000,0.0,5.468232,7.611178,NE,24.00000,SE,...,9.0,45.0,16.0,1017.6,1012.8,4.447461,4.50993,18.1,26.5,No
4,2008-12-05,Albury,17.5,32.300000,1.0,5.468232,7.611178,W,41.00000,ENE,...,20.0,82.0,33.0,1010.8,1006.0,7.000000,8.00000,17.8,29.7,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145455,2017-06-21,Uluru,2.8,23.400000,0.0,5.468232,7.611178,E,31.00000,SE,...,11.0,51.0,24.0,1024.6,1020.3,4.447461,4.50993,10.1,22.4,No
145456,2017-06-22,Uluru,3.6,25.300000,0.0,5.468232,7.611178,NNW,22.00000,SE,...,9.0,56.0,21.0,1023.5,1019.1,4.447461,4.50993,10.9,24.5,No
145457,2017-06-23,Uluru,5.4,26.900000,0.0,5.468232,7.611178,N,37.00000,SE,...,9.0,53.0,24.0,1021.0,1016.8,4.447461,4.50993,12.5,26.1,No
145458,2017-06-24,Uluru,7.8,27.000000,0.0,5.468232,7.611178,SE,28.00000,SSE,...,7.0,51.0,24.0,1019.4,1016.5,3.000000,2.00000,15.1,26.0,No


In [6]:
y = df_weather[target]
y


y[y == 'No'] = 0
y[y == 'Yes'] = 1

y = pd.to_numeric(y, errors='coerce')
y

C:\Users\Gabriel\AppData\Local\Temp/ipykernel_27536/1277270351.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y[y == 'No'] = 0
C:\Users\Gabriel\AppData\Local\Temp/ipykernel_27536/1277270351.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y[y == 'Yes'] = 1


0         0
1         0
2         0
3         0
4         0
         ..
145455    0
145456    0
145457    0
145458    0
145459    0
Name: RainTomorrow, Length: 145460, dtype: int64

In [24]:
class Tree_c45:
    def __init__(self):
        # primero indicamos si el objeto es hoja o raiz
        self.is_leaf = True
        
        # atributos para el corte cuando el objeto es una raiz
        # indicamos el indice de la variable de corte y los valores
        self.var_index = -1
        self.cut_value = 0
        # los hijos deben ser objetos de Tree_c45
        self.list_of_childs = []
        
        # atributos cuando el Tree_c45 es una hoja
        # indicamos el valor de la clase y en class_count una tupla (casos con ese valor, casos totales en la hoja)
        self.class_value = -1
        self.class_count = 0
        
        # para el nivel de profundidad en el arbol de la raiz o la hoja
        self.level = -1
        
    def __str__(self):
        output = ''
        if(self.is_leaf):
            output += 'Class value: ' + str(self.class_value) + '\tCounts: ' + str(self.class_count)
        else:
            output += 'Feature '+ str(self.var_index)
            for i in range(len(self.list_of_childs)):
                output += '\n'+'\t'*(self.level+1)+str(self.cut_value[i])+': '+str(self.list_of_childs[i]) 
            
        return output
    
    # esta funcion nos servira para hacer predicciones y debe ser completada
    def predict(self,x):
        # TO DO
        #Crear dos arboles nuevos
        if (self.is_leaf):
            return self.class_value
        for child in self.list_of_childs:
            if x[self.var_index] == child.cut_value:
                return child.predict(x)

        return

In [25]:
from scipy.stats import entropy
from sklearn.base import BaseEstimator

class C45(BaseEstimator):
    # constructor con la profundidad del arbol, que por defecto es 2 y donde inicializamos el arbol que vamos a aprender
    def __init__(self, max_depth):
        
        self.max_depth = max_depth
        self.tree = Tree_c45()
        
    def __init__(self):
    
        self.max_depth = 2
        self.tree = Tree_c45()
    #siguiendo las guias de scikit-learn, creamos las funciones fit para el entrenamiento
    def fit(self, X, y):
        # esta será la llamada a la funcion recursiva que aprendera el arbol a partir de los datos
        self.make_cut(X, y, self.tree, 0)
        
    # y predict para las predicciones de casos nuevos
    def predict(self, X):
        t = []
        for i in range(len(X)):
            t.append(self.tree.predict(X.iloc[i]))
        #return np.array([self.tree.predict(x) for x in X])
        return t
    
    # tambien se da una funcion score que calcula el accuracy
    def score(self, X, y):
        return np.sum(self.predict(X)==y)/y.shape[0]
    
    # esta es la funcion principal, encargada de encontrar la variable y su punto de corte que maximice la ganancia en base a la entropia.
    # se basa en que los cortes se van a realizar sobre variables continuas aunque la salida sea discreta
    def make_cut(self, X, y, current_tree, current_depth):
        

        # esqueleto con la funcionalidad basica
        
        # antes de realizar ninguna operacion comprobamos si hemos alcanzado el limite del arbol,
        # si es asi, actualizamos el arbol ya que estamos en un nodo hoja 
        # y no hace falta buscar variable ni continuar con las llamadas recursivas
        if current_depth >= self.max_depth:
            # TO DO

            #current_tree.is_leaf = ...
            current_tree.is_leaf = True
            #current_tree.class_value = ...
            if y.value_counts()[0] > y.value_counts()[1]:
                current_tree.class_value = 0
                current_tree.class_count = y.value_counts()[0]
            else: 
                current_tree.class_value = 1
                current_tree.class_count = y.value_counts()[1]
            return 

        # primero, obtenemos la entropia de la clase
        # TO DO        
        class_entropy = self.entropy(y)
        # segundo, deberemos recorrer todas las caracteristicas siguiendo estos pasos:
        # 1. Ordenar los valores de la caracteristica y aplicar a la clase
        
        #Declaramos las variables donde guardaremos los datos de por donde cortar el arbol
        finalinfogain = 0
        finalarraytablas = []
        finalarraytargets = []
        finalcol= None
        finalval = None
        #Tenemos que mirar todas las columnas para saber cual es el mejor corte
        for col in X.columns: 
            #Creamos un dataframe auxiliar, t, en donde estarán los datos ordenados
            t = pd.concat([X,y], axis =1)
            t = t.sort_values(col)
            features = t.columns
            attributes = features[:-1]
            target = features[-1]
            

            X = t[attributes]
            y = t[target]
            
            
            #Aquí tenemos que diferenciar las variables categóricas de las numéricas.
            #Para no depender de los índices de los dataframes, convertimos la columna a un array
            auxilio = np.array(X[col])
            #Y obtenemos el tipo del primer elemento de ese array
            # 2. Obtener los puntos de corte en base a la clase ordenada
            if str == type(auxilio[0]):
                #En caso de ser variable catégorica (el tipo sea igual a string), declaramos variables auxiliares
                arraytablas = []
                arraytargets = []
                arrayentropia = []
                arrayprob = []
                #Y para cada valor unico de la columna, creamos una tabla y calculamos su entropía
                for label in X[col].unique():
                    t1 = t[t[col] == label]
                    X1 = t1[t1.columns[:-1]]
                    
                    #La tabla la guardamos para despúes, en caso de tener que cortar por aquí, poder pasarsela a la siguiente llamada de make_cut
                    arraytablas.append(X1)
                    
                    
                    y1 = t1[t1.columns[-1]]
                    arraytargets.append(y1)
                    
                    # 3.Calculamos la entropia y la probabilidad de ramificar por ahí
                    e1 = self.entropy(y1)
                    arrayentropia.append(e1)
                    p1 = np.sum(y1.value_counts())/(np.sum(y.value_counts()))
                    arrayprob.append(p1)
                    
                 # 4. Calculamos la ganancia con la entropia obtenida y la de la clase
                sum_entropy = 0
                for i in range(0,len(arrayprob)):
                    sum_entropy += arrayprob.pop() * arrayentropia.pop()
                infogain = class_entropy - sum_entropy 
                    
                # 5. Almacenamos la variable y el punto de corte con mayor ganancia, así como las tablas generadas y sus targets 
                if finalinfogain < infogain:
                    finalinfogain = infogain
                    finalcol = col
                    finalarraytablas = arraytablas.copy()
                    finalarraytargets= arraytargets.copy()
                
            else:
                #En caso de ser numérica, transformamos la columna en un array para iterar por el
                vectorCol = np.array(X[col])
                vectorTarget = np.array(y)
                #Y guardamos el valor inicial del vector target 
                actual = vectorTarget[0]
                #Iteramos por todo el vector buscando en el que el valor i del vector sea diferente al actual, y difente del siguiente también
                for i in range(0,len(vectorCol)-1):
                    if (actual != vectorTarget[i]) & (actual != vectorTarget[i+1]):
                        
                        #Cambiamos el valor actual
                        actual = vectorTarget[i]

                        #Y guardamos las secciones del target y de la tabla
                        t1 = vectorCol[:i]  
                        y1 = vectorTarget[:i]
                        
                        t2 = vectorCol[i:]
                        y2 = vectorTarget[i:]
                        
                        #Calculamos la entropía y el infogain
                        e1 = self.entropy(pd.DataFrame(y1))
                        p1 = len(y1)/len(vectorTarget)
                                                            
                                    
                        e2 = self.entropy(pd.DataFrame(y2))
                        p2 = len(y2)/len(vectorTarget)
                              
                        
                        infogain = class_entropy - e1*p1 - e2*p2 
                            
                         # 5. Almacenamos la variable y el punto de corte con mayor ganancia, al igual que las tablas de target y atributos
                        if finalinfogain < infogain:
                            finalinfogain = infogain
                            finalcol = col
                            finalval = vectorCol[i]
                            
                            aux = t[t[col] <= finalval]
                            X1 = aux[aux.columns[:-1]]
                            y1 = aux[aux.columns[-1]]
                            #Antes de hacer el append, vaciamos las variables de anteriores iteraciones
                            finalarraytablas = []
                            finalarraytargets = []
                            
                            finalarraytablas.append(X1)
                            finalarraytargets.append(y1)
                            
                            aux = t[t[col] > finalval]
                            X2 = aux[aux.columns[:-1]]
                            y2 = aux[aux.columns[-1]]

                            
                            finalarraytablas.append(X2)
                            finalarraytargets.append(y2)
                
    
           
            # TO DO
            
            
        # antes de llamar a la funcion recursiva hay que actualizar los valores del arbol
        # TO DO
        #current_tree.is_leaf = ...
        current_tree.is_leaf = False
        #current_tree.level = ...
        current_tree.level = current_depth
        #current_tree.var_index = ...
        current_tree.var_index = finalcol
        #current_tree.cut_value = ...
        #El valor de corte va a ser distinto de un corte númerico (menor o igual y mayor), o corte categórico (cada valor único que tome la columna)
        auxilio = np.array(X[finalcol])
        if str == type(auxilio[0]):
            current_tree.cut_value = X[finalcol].unique()
            for i in X[finalcol].unique():
                #Y creamos un número de hijos igual al número de valores de corte
                current_tree.list_of_childs.append(Tree_c45())
                
        else:
            current_tree.cut_value = finalval
            
            current_tree.list_of_childs.append(Tree_c45())
            current_tree.list_of_childs.append(Tree_c45())
        
        # para continuar con el algoritmo hay que hacer una llamada a la funcion
        # de forma recursiva para cada hijo
        for i in current_tree.list_of_childs:
            
            xi = pd.DataFrame(finalarraytablas.pop())
            yi = pd.DataFrame(finalarraytargets.pop())            
            
            self.make_cut(xi, yi, i, current_tree.level+1)
        return 0
    
    #Definimos la función entropía que se usa en make_cut
    def entropy(self,y):
        a = y.value_counts()/y.shape[0]
        entropy = np.sum(-a*np.log2(a))
        return(entropy)
    

    

In [26]:
def entropy(y):
        a = y.value_counts()/y.shape[0]
        entropy = np.sum(-a*np.log2(a))
        return(entropy)

prueba = pd.concat([df_weather['Humidity3pm'],df_weather['Sunshine']], axis =1)
prueba = prueba.sort_values('Humidity3pm')
algoritmo = C45()




features = df_weather.columns
attributes = features[1:-1]
attributes1 =['Location','Humidity3pm','Sunshine']
target = features[-1]

X = df_weather[attributes1]



y = df_weather[target]

y[y == 'No'] = 0
y[y == 'Yes'] = 1

y = pd.to_numeric(y, errors='coerce')

algoritmo.fit(X,y)

C:\Users\Gabriel\AppData\Local\Temp/ipykernel_27536/2600692761.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y[y == 'No'] = 0
C:\Users\Gabriel\AppData\Local\Temp/ipykernel_27536/2600692761.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y[y == 'Yes'] = 1


## Notas

El arbol es muy poco eficiente, hemos probado a reducir el tamaño de la base de datos para ver si funciona bien. Creemos que va bien pero demasiado lento, y que no lo hemos probado sobre todo porque tardaba demasiado.

Además el predict nos funciona mal porque no guardamos bien el class_value, pero tampoco sabemos porque.